## Launch an Inference Server (vLLM) for the Compressed Model

This step sets up a vLLM inference server to host your compressed model and exposes an OpenAI-compatible API endpoint. This server is required so that GuideLLM can benchmark system-level performance like throughput, latency, and time-to-first-token.

**Goal**: Make the compressed model accessible via an API for performance evaluation.

**Output**: vLLM server running with the compressed model, ready to accept requests.

**Resources used** : 46GB L40S GPU x 1


In [ ]:
import os

from utils import generate, stream

In [ ]:
# set the logging level for vLLM inference
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

### vLLM config for single node
For **single-node**, **single-GPU** or **multi-GPU (but not multinode)** vLLM serving, the main arguments are:

``--model``: Model path or Hugging Face repo ID (required).

``--tensor-parallel-size``: Number of GPUs to use (set to 1 for single GPU, or >1 for multi-GPU tensor parallelism).

``--port``: Port for the API server (default is 8000).

``--host``: Host IP address (default is 127.0.0.1).

``--gpu-memory-utilization``: controls what fraction of each GPU’s memory vLLM will use for the model executor and KV cache. For example, --gpu-memory-utilization 0.5 means vLLM will use 50% of the GPU memory.

``--quantization``: Method used to quantize the weights. 
 
``--max-model-len``: argument sets the maximum context length (in tokens) that vLLM can process for both prompt and output combined. If not specified, it defaults to the model’s config value. Setting a higher value allows longer prompts/completions but increases GPU memory usage for the KV cache; setting it lower saves memory but limits context length. Set this to prevent problems with memory if the model’s default context length is too long.

For **multi-node** vLLM serving, use:

 ``--tensor-parallel-size`` Number of GPUs per node (or total GPUs if not using pipeline parallelism).
 
``--pipeline-parallel-size`` Number of nodes (optional, for pipeline parallelism).

Additionally, for multi-node setup, a Ray cluster is also needed.



# Run this command in terminal to serve the compressed model using vLLM

Make sure you are in the parent directory of compressed_model.

```
vllm serve \
  "Llama_3.1_8B_Instruct_int8_dynamic" \
  --host 127.0.0.1 \
  --port 8000 \
  --gpu-memory-utilization 0.6 \
  --tensor-parallel-size 1 \
  --pipeline-parallel-size 1 \
  --max-model-len 2048
```

Once the server starts, you will see something like this:

```INFO:     Started server process [37883]```\
```INFO:     Waiting for application startup.```\
```INFO:     Application startup complete.```


## A test run to see if the vLLM server is accessible
We use a helper function **generate** (defined in [utils.py](./utils.py)) to simplify sending requests to our locally-served VLLM model.
This function wraps the OpenAI-compatible Chat Completions API exposed by VLLM.
### Why we Use the OpenAI SDK with vLLM
vLLM implements an OpenAI-compatible REST API, meaning:

- vLLM starts a local web server (e.g., http://127.0.0.1:8000/v1)

- it exposes the same endpoints (/v1/chat/completions, /v1/completions) as OpenAI

- accepts the same request schema (messages=[{"role": "..."}])

- the same client interface as OpenAI

- The OpenAI SDK doesn't know (or care) that it isn’t talking to OpenAI — it just sends HTTP requests to the specified url in the expected format

An **alternate** way is to send POST requests using python's **requests** module.


In [ ]:
# For non streaming results
response = generate(
    model="Llama_3.1_8B_Instruct_int8_dynamic",
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8000,
    api_key="empty",
    max_tokens=512,
)
print(response)

Photosynthesis is a vital process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose. This process is essential for life on Earth, as it provides the primary source of energy for nearly all living organisms.

During photosynthesis, plants use energy from sunlight to convert carbon dioxide (CO2) and water (H2O) into glucose (a type of sugar) and oxygen (O2). This process involves several stages, including:

1. Light absorption: Plants absorb light energy from the sun using pigments such as chlorophyll, which is present in chloroplasts, the organelles responsible for photosynthesis.
2. Light-dependent reactions: The absorbed light energy is used to generate ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleotide phosphate), which are high-energy molecules that will be used in the next stage of photosynthesis.
3. Light-independent reactions (Calvin cycle): In this stage, CO2 is fixed into glucose using 

In [ ]:
# For streaming results
res = ""
for chunk in stream(
    model="Llama_3.1_8B_Instruct_int8_dynamic",
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8000,
    api_key="empty",
    max_tokens=512,
):
    res += chunk
    print(chunk, end="", flush=True)

Photosynthesis is a vital biological process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. This process is essential for life on Earth, as it provides the energy and organic compounds needed to support the food chain.

During photosynthesis, plants use energy from sunlight, water, and carbon dioxide to produce glucose and oxygen. The overall equation for photosynthesis is:

6 CO2 (carbon dioxide) + 6 H2O (water) + light energy → C6H12O6 (glucose) + 6 O2 (oxygen)

Here's a simplified overview of the photosynthetic process:

1. **Light absorption**: Plants absorb light energy from the sun through specialized pigments such as chlorophyll.
2. **Water absorption**: Plants absorb water from the soil through their roots.
3. **Carbon dioxide absorption**: Plants absorb carbon dioxide from the air through tiny openings on their leaves called stomata.
4. **Light-dependent reactions**: The absorb

In [ ]:
print(res)

Photosynthesis is a vital biological process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. This process is essential for life on Earth, as it provides the energy and organic compounds needed to support the food chain.

During photosynthesis, plants use energy from sunlight, water, and carbon dioxide to produce glucose and oxygen. The overall equation for photosynthesis is:

6 CO2 (carbon dioxide) + 6 H2O (water) + light energy → C6H12O6 (glucose) + 6 O2 (oxygen)

Here's a simplified overview of the photosynthetic process:

1. **Light absorption**: Plants absorb light energy from the sun through specialized pigments such as chlorophyll.
2. **Water absorption**: Plants absorb water from the soil through their roots.
3. **Carbon dioxide absorption**: Plants absorb carbon dioxide from the air through tiny openings on their leaves called stomata.
4. **Light-dependent reactions**: The absorb

### Checking GPU vRAM
Loading the compressed model with the congfiguration defined in the above command will take approximately 28.5GB. It may seem surprising that a compressed 8.5 GB model consumes ~28.5 GB GPU memory. This is expected behavior in vLLM, due to how memory is allocated during inference. The main contributors are:

1. **Model Weights (~8.5 GB)**

    The size of your compressed model stored on disk (INT8, FP16, etc.). 
    Loaded once into GPU memory.
   
2. **Runtime GPU Memory (~6 GB)**

- vLLM reserves extra memory for:

- Parameter sharding

- CUDA kernels

- Attention buffers and temporary tensors

- Weight adapters and padded tensors

This adds ~4–8 GB depending on the model.

3. **KV Cache (~14 GB)**

 - Stores key/value tensors for each generated token to avoid recomputation.

- Memory grows with sequence length, model hidden size, and concurrency.

- vLLM presets a large KV cache to support batching efficiently.


4. **GPU Memory Utilization Flag (--gpu-memory-utilization)**

``--gpu-memory-utilization`` is set to 0.6, meaning vLLM can utilize 60% of the total GPU memory. In this case, we have used one 46GB LS40 GPU, 60% of 46 is approx 28.